In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import math
import shutil
import random
import torch

In [5]:
torch.__version__

'1.8.1'

In [2]:
def process_hdr(image):
    return np.minimum(np.maximum(np.log10(image * 0.2 * image.mean() + 1), 0), 2) - 1

def find_power(image, target=0.011):
    l, r = 0.0, 1.0
    m = 0.5
    for i in range(32):
        res = np.power(image, m).mean()
        if abs(res - target) < 1e-6:
            return m
        elif res > target:
            l = m
        else:
            r = m
        m = (r + l) / 2
    return m

In [3]:
src_dir = os.path.join('LavalIndoorHDRDatasetPrepared')
dst_dir = os.path.join('LavalIndoorHDRDatasetReadySmall')

catalog = os.listdir(src_dir)
n = len(catalog)
for i, file in enumerate(catalog):
    data = cv2.resize(np.load(os.path.join(src_dir, file))[:3300,], (256, 128))
    data /= data.max()

    target = np.random.rand() * (0.015 - 0.011) + 0.011
    data = np.power(data, find_power(data, target)) * 255
    data = process_hdr(data)

    np.save(os.path.join(dst_dir, file[:-4]), data)
    print('{} / {}'.format(i + 1, n), end='\r')

In [7]:
src_dir = os.path.join('PanoIndoorLDRDataset')
dst_dir = os.path.join('PanoIndoorLDRDatasetReadySmall')

catalog = os.listdir(os.path.join(src_dir, 'bedroom'))
n = len(catalog) - 2
delay = 0
for i, file in enumerate(catalog):
    if file.endswith('.mat'):
        delay += 1
        continue

    data = cv2.imread(os.path.join(src_dir, 'bedroom', file, file + '.jpg'))
    data = cv2.resize(data, (256, 128))
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    data = process_hdr(data)

    np.save(os.path.join(dst_dir, file), data)
    print('{} / {}'.format(i + 1 - delay, n), end='\r')

print()

catalog = os.listdir(os.path.join(src_dir, 'living_room'))
n = len(catalog) - 2
delay = 0
for i, file in enumerate(catalog):
    if file.endswith('.mat'):
        delay += 1
        continue

    try:
        data = cv2.imread(os.path.join(src_dir, 'living_room', file, file + '.jpg'))
        data = cv2.resize(data, (256, 128))
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
        data = process_hdr(data)
    except:
        delay += 1

    np.save(os.path.join(dst_dir, file), data)
    print('{} / {}'.format(i + 1 - delay, n), end='\r')

In [9]:
for file in os.listdir('LavalIndoorHDRDatasetReadySmall'):
    if file in ('test', 'train'):
        continue
    shutil.move(
        os.path.join('LavalIndoorHDRDatasetReadySmall', file),
        os.path.join('LavalIndoorHDRDatasetReadySmall', 'train' if random.random() < 0.85 else 'test', file)
    )
for file in os.listdir('PanoIndoorLDRDatasetReadySmall'):
    if file in ('test', 'train'):
        continue
    shutil.move(
        os.path.join('PanoIndoorLDRDatasetReadySmall', file),
        os.path.join('PanoIndoorLDRDatasetReadySmall', 'train' if random.random() < 0.85 else 'test', file)
    )